# Set Imports & Configurations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# This line will install the spotify API if not installed yet.
!pip install spotipy

In [ ]:
# All imports and configurations needed
import spotipy, pandas as pd, os, json, sys, timeit, matplotlib.pyplot as plt
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import Image
import urllib.request

CLIENT_ID = '0334096aa9004802afe7d5caf79aeb95'
CLIENT_SECRET = 'd5e962e82f834f068269378209c5f83e'

base_dir = '/content/drive/My Drive/BUAN6505 Final Project Folder/Data'

token = spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  del sys.path[0]


#Get Metadata by Year

In [ ]:
"""
Input:
  - List of years to pull songs from
Returns:
  - DataFrame with song meta data (including Popularity) for each year passed
  - Please see Data Dictionary for definition of fields
"""
def get_yearly_data(year_lst):
  if not isinstance(year_lst,list):
    year_lst = [year_lst]

  results_df = pd.DataFrame()
  audio_features_df = pd.DataFrame()
  for year in year_lst:
    for i in range(0,1500,50):
      try:
        track_results = sp.search(q='year:' + str(year), type='track', limit=50,offset=i)

        for i, t in enumerate(track_results['tracks']['items']):
            results_df = results_df.append({'artist':t['artists'][0]['name'],
                                            'track_name':t['name'],
                                            'id':t['id'],
                                            'popularity':t['popularity'],
                                            'year':str(year),
                                            'genre':sp.artist(t["artists"][0]["external_urls"]["spotify"])['genres'][0]},
                                            ignore_index=True)
            # Get audio features
            audio_features = sp.audio_features(t['id'])[0]
            for k,v in audio_features.items():
              audio_features[k] = v 
            audio_features_df = audio_features_df.append(audio_features,ignore_index=True)
      except:
        pass
  results_df = results_df.merge(audio_features_df,on='id',how='left')
  return results_df



In [ ]:
# Start timer
start = timeit.default_timer()

# Run function
data_pull = get_yearly_data(list(range(2008,2021)))

# Stop timer
stop = timeit.default_timer()
print('Time to run this code (in seconds):', round(stop - start,2))

In [ ]:
# Send to csv
# data_pull.to_csv(os.path.join(base_dir,'Playlist_Data_testing.csv'),index=False)